In [25]:
import logging
from enum import Enum, auto
from pathlib import Path
import shutil

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import webcolors

from dsgrid.dataformat.datafile import Datafile
from dsgrid.dataformat.datatable import Datatable
from dsgrid.dataformat.enumeration import census_divisions, conus, conus_states, enumdata_folder
from dsgrid.dataformat.dimmap import mappings
from dsgrid.helpers import ensure_enum, lighten_color, palette
from dsgrid.model import ComponentType, LoadModelComponent, LoadModel

enumdata_folder = Path(enumdata_folder)

from ntbkhelp import OptionPresenter, show_enum, show_elements_with_data

dsgrid_nrel_base_path = Path("//nrelnas01/PLEXOS/Projects/Load/dsgrid_v0.2.0/")
dsgrid_oedi_base_path = None

logger = logging.getLogger(__name__)

# USER INPUT ----------------------------------------
# Choose which base path you would like to use
dsgrid_base_path = dsgrid_nrel_base_path
# ---------------------------------------------------

class AnalysisType(Enum):
    Residuals = auto() # modeled components, historical data, and residuals
    Modeled = auto()   # only dsgrid-modeled "bottom-up" and "gap" components

all_components_model_path = dsgrid_base_path / "products" / "state_hourly_residuals"
bottom_up_components_model_path = dsgrid_base_path / "products" / "dsgrid_site_energy_state_hourly"

# USER INPUT ----------------------------------------
#
# Specify which version of dsgrid you want to examine
analysis_type = AnalysisType.Modeled
#
# Specify a local directory for any outputs
output_dir = Path.home() / "Documents" / "dsgrid-legacy-efs"
#
# Plotting code will always overwrite graphics (e.g., 
# .png files) on disk
# 
# If you also want to re-process dsgrid model mappings 
# whenever you run a cell containing such work, 
# set overwrite to True
overwrite = False
#
# Specify logging level
log_level = logging.INFO
# ---------------------------------------------------

logging.basicConfig(level=log_level)

analysis_type = ensure_enum(AnalysisType, analysis_type)
dsgrid_dataset_path = all_components_model_path if analysis_type == AnalysisType.Residuals else bottom_up_components_model_path

def make_dir(p, prompt=True):
    if p.exists():
        return
    if prompt:
        input_str = input(f"{p!r} does not exist. Would you like to create it? [Y/n] ")
        if not (input_str[0].lower() == "y"):
            return
    if not p.parent.exists():
        make_dir(p.parent, prompt=prompt)
    p.mkdir()
    print(f"Created {p!r}.")

make_dir(output_dir)

In [19]:
## Helper functions and data for setting up dsgrid models

def load_components(datadir,component_type,tuple_list):
    result = []
    result = []
    for name, color, filepath in tuple_list:
        if not (datadir / filepath).exists():
            logger.info(f'No {name} component in {datadir}')
            continue
        result.append(LoadModelComponent(name,component_type=component_type,color=color))
        result[-1].load_datafile(datadir / filepath)
    return result

# Bottom-Up
bottomup_components_list = [
    ('Residential','#F7A11A','residential.dsg'),
    ('Commercial','#5D9732','commercial.dsg'),
    ('Industrial','#D9531E','industrial.dsg')]

# Gaps
color_fade_frac = 0.3
gap_components_list = [
    ('Residential Gaps',lighten_color(bottomup_components_list[0][1],color_fade_frac),'residential_gaps.dsg'),
    ('Commercial Gaps',lighten_color(bottomup_components_list[1][1],color_fade_frac),'commercial_gaps.dsg'),
    ('Industrial Gaps',lighten_color(bottomup_components_list[2][1],color_fade_frac),'industrial_gaps.dsg'),
    ('Transportation Gaps',lighten_color("#0079C2",color_fade_frac),'trans_rail_hourly.dsg'),
    ('Municipal Water','#00CCCC','municipal_water.dsg'),
    ('Outdoor Lighting','#FFFF33','outdoor_lighting.dsg')]

# DG
dg_components_list = [
    ('CHP and Thermal DG','#99004D','chp_dg.dsg'),
    ('Distributed PV','#CD9B1D','distributedpv_sectoral.dsg'),
    ('Distributed Generation','#99004D','distributed_generation.dsg')]

# Derived
derived_components_list = [
    ('Loss Model','#9A9A9A','loss_model.dsg'),
    ('Hourly Residuals','#632E86','hourly_residuals.dsg')]

# Top-Down
topdown_components_list = [
    ('Top-Down Hourly',None,'historical_hourly_load.dsg'),
    ('Top-Down Annual Energy',None,'eia_annual_energy_by_sector.dsg')]

# informational list of components which use sector_id to differentiate between res, com, ind
# other components either use sector_id to represent subsectors or have a single sector_id
by_sector_components = [
    (ComponentType(ComponentType.TOPDOWN),'Top-Down Annual Energy'),
    (ComponentType(ComponentType.DG),'CHP and Thermal DG'),
    (ComponentType(ComponentType.DG),'Distributed PV')
]

def get_model(model_dir):
    bottomup_components = load_components(model_dir,ComponentType.BOTTOMUP,bottomup_components_list)
    gap_components = load_components(model_dir,ComponentType.GAP,gap_components_list)
    dg_components = load_components(model_dir,ComponentType.DG,dg_components_list)
    derived_components = load_components(model_dir,ComponentType.DERIVED,derived_components_list)
    topdown_components = load_components(model_dir,ComponentType.TOPDOWN,topdown_components_list)
    return LoadModel.create(bottomup_components + gap_components + dg_components + derived_components + topdown_components)

def downselect_model(model,component_keys):
    """
    Parameters
    ----------
    model : dsgrid.LoadModel
    component_keys : list of keys in model.components
        List of component keys in model that are to be KEPT.
        
    Returns
    -------
    dsgrid.LoadModel
        Same object as model. Upon return any components whose keys are not in 
        component_keys will have been deleted.
    """
    to_delete = []
    for key in model.components:
        if key not in component_keys:
            to_delete.append(key)
    for key in to_delete:
        del model.components[key]
    return model

In [29]:
## Helper plotting functions

from ntbkhelp import model_tz, get_dim_order, add_temporal_category, clean_name


def seasonal_example_weeks_sectors(model, geo_id=None):
    """
    dsgrid Model Documentation Figure 19
    """
    pass

def seasonal_example_weeks_residuals(model, geo_id=None):
    """
    dsgrid Model Documentation Figure 20
    """
    pass

def seasonal_diurnal_profiles(model, component_id, geo_id=None, plots_dir=None, show=False, 
                              subsector_plot=True, enduse_plot=True, enduse_by_subsector_plots=True,
                              area_plots=True, line_plots=True):
    """
    dsgrid Model Documentation Figures 23, 24, 25
    """
    plot_kwargs = dict(
        font_size = 12,
        figsize = (6.5,6.5),
        n_leg_cols = 1,
        leg_pos = 5,
        axis_position_kwargs = {
            'ax_lowleft_pos': '0.115,0.075',
            'ax_upright_pos': '0.675,0.95'
        },
        legend_kwargs = {
            'labelspacing': 0.02,
            'handleheight': 1.2,
            'handlelength': 1.2,
            'ncol': n_leg_cols,
            'fontsize': font_size-1
        },
        subplots_adjust_kwargs = {
            'hspace': 0.001,
            'wspace': 0.001
        },
        max_subsectors = 15)
    
    plt.style.use(['ggplot'])

    # Get the data to plot
    data = model[component_id].get_datatable(sort=False,verify_integrity=False).data
    if geo_id is not None:
        logger.info(f"Selecting {geo_id} data from {component_id}")
        data = data[:,geo_id,:,:]
    enduses = get_dim_order(data,'enduse'); enduse_enum = model[component_id].datafile.enduse_enum
    logger.info(f"Plotting profiles for {component_id} with enduses {enduses}")
    subsectors = get_dim_order(data,'sector'); sector_enum = model[component_id].datafile.sector_enum
    geographies = get_dim_order(data,'geography'); geo_enum = model[component_id].datafile.geo_enum

    # color palettes
    enduse_colors = palette(model[component_id].color,len(enduses))
    subsector_colors = palette(model[component_id].color,len(subsectors))

    # add temporal information -- season, weekday/weekend, hour 1 to 24
    data.name = 'value'
    data = data.reset_index()
    season_map = pd.read_csv(enumdata_folder / 'hourly2012_to_seasons.csv')
    daytype_map = pd.read_csv(enumdata_folder / 'hourly2012_to_daytypes.csv')
    hours_map = pd.read_csv(enumdata_folder / 'hourly2012_to_hours.csv')
    data = add_temporal_category(data,season_map,'season')
    data = add_temporal_category(data,daytype_map,'day_type')
    data = add_temporal_category(data,hours_map,'hour')
    
    tmp = data.set_index(['sector','geography','enduse','time','season','day_type','hour'])
    tmp = pds.Series(tmp['value'])

    # refresh just for this dataset
    enduses = get_dim_order(tmp,'enduse')
    subsectors = get_dim_order(tmp,'sector')
    enduse_colors = []; tmp = palette(model[key].color,len(enduses))
    for i, enduse in enumerate(enduses):
        if enduse in enduse_color_map:
            enduse_colors.append(enduse_color_map[enduse])
        else:
            enduse_colors.append(tmp[i])
    subsector_colors = palette(model[key].color,len(subsectors))
    
    component_prefix = component_id[1].lower().replace(' ','-')

    if subsector_plot:
        # plot by subsector
        to_plot = data.groupby(['time','season','day_type','hour','sector']).sum().reset_index()
        to_plot = to_plot.pivot_table(values='value',
                                      index=['season','day_type','hour'],
                                      columns='sector',
                                      aggfunc=np.mean)
        to_plot.columns = [col for col in to_plot.columns]
        to_plot = to_plot[subsectors]      

        if len(subsectors) > max_subsectors:
            to_collapse = subsectors[max_subsectors:]
            to_plot['Other'] = to_plot[to_collapse].sum(axis=1)
            for col in to_collapse:
                del to_plot[col]
            subsector_colors = palette(model[key].color,max_subsectors+1)

        component_plot(to_plot,subsector_colors,sector_enum,
                       plots_dir / f'{component_prefix}-subsectors-area.png' if (plots_dir is not None) and area_plots else None,
                       plots_dir / f'{component_prefix}-subsectors-line.png' if (plots_dir is not None) and line_plots else None,
                       show=show, **plot_kwargs)

    # if multiple end-uses
    if len(enduses) > 1:
        if enduse_plot:
            # plot by enduse
            to_plot = data.groupby(['time','season','day_type','hour','enduse']).sum().reset_index()
            to_plot = to_plot.pivot_table(values='value',
                                          index=['season','day_type','hour'],
                                          columns='enduse',
                                          aggfunc=np.mean)
            to_plot.columns = [col for col in to_plot.columns]
            to_plot = to_plot[enduses]                
            component_plot(to_plot,enduse_colors,enduse_enum,
                           plots_dir / f'{component_prefix}-enduses-area.png' if (plots_dir is not None) and area_plots else None,
                           plots_dir / f'{component_prefix}-enduses-line.png' if (plots_dir is not None) and line_plots else None,
                           show=show, **plot_kwargs)

        # if multiple subsectors and end-uses
        if (len(subsectors) > 1) and (len(subsectors) < 30) and enduse_by_subsector_plots:
            # for each subsector
            for subsector in subsectors:
                # plot by enduse
                to_plot = data[data.sector == subsector].groupby(['time','season','day_type','hour','enduse']).sum().reset_index()
                to_plot = to_plot.pivot_table(values='value',
                                              index=['season','day_type','hour'],
                                              columns='enduse',
                                              aggfunc=np.mean)
                to_plot.columns = [col for col in to_plot.columns]
                to_plot = to_plot[enduses]
                subsector_prefix = clean_name(sector_enum.get_name(subsector)).lower().replace(' ', '-')
                component_plot(to_plot,enduse_colors,enduse_enum,
                               plots_dir / f'{component_prefix}-{subsector_prefix}-enduses-area.png' if (plots_dir is not None) and area_plots else None,
                               plots_dir / f'{component_prefix}-{subsector_prefix}-enduses-line.png' if (plots_dir is not None) and line_plots else None,
                               show=show, **plot_kwargs)
    
    return data

## Contiguous United States

In [6]:
# Create the CONUS model if it does not exist or overwrite is True
model_name = 'conus_hourly_residuals' if analysis_type == AnalysisType.Residuals else "dsgrid_site_energy_conus_hourly"
model_path = output_dir / model_name
if model_path.exists() and overwrite:
    logger.info(f"Deleting {model_path} and re-creating. If this "
                "is not the desired behavior, set overwrite = False.")
    shutil.rmtree(model_path)

if not model_path.exists():
    model = get_model(dsgrid_dataset_path)
    model.map_dimension(model_path, conus, mappings)
model = get_model(model_path)

INFO:__main__:No Residential Gaps component in C:\Users\ehale\Documents\dsgrid-legacy-efs\dsgrid_site_energy_conus_hourly
INFO:__main__:No CHP and Thermal DG component in C:\Users\ehale\Documents\dsgrid-legacy-efs\dsgrid_site_energy_conus_hourly
INFO:__main__:No Distributed PV component in C:\Users\ehale\Documents\dsgrid-legacy-efs\dsgrid_site_energy_conus_hourly
INFO:__main__:No Loss Model component in C:\Users\ehale\Documents\dsgrid-legacy-efs\dsgrid_site_energy_conus_hourly
INFO:__main__:No Hourly Residuals component in C:\Users\ehale\Documents\dsgrid-legacy-efs\dsgrid_site_energy_conus_hourly
INFO:__main__:No Top-Down Hourly component in C:\Users\ehale\Documents\dsgrid-legacy-efs\dsgrid_site_energy_conus_hourly
INFO:__main__:No Top-Down Annual Energy component in C:\Users\ehale\Documents\dsgrid-legacy-efs\dsgrid_site_energy_conus_hourly


In [ ]:
# seasonal_example_weeks_sectors, e.g., dsgrid Model Documentation Figure 19
assert analysis_type == AnalysisType.Residuals, f"Wrote analysis_type {analysis_type} for this plot"

In [ ]:
# seasonal_example_weeks_residuals, e.g., dsgrid Model Documentation Figure 20
assert analysis_type == AnalysisType.Residuals, f"Wrote analysis_type {analysis_type} for this plot"

In [30]:
# seasonal_diurnal_profiles, e.g., dsgrid Model Documentation Figures 23, 24, 25
assert analysis_type == AnalysisType.Modeled, f"Wrote analysis_type {analysis_type} for this plot"

# USER INPUT ----------------------------------------
# 
# Whether to show plots in this notebook
show = True
# Output directory for .png files -- set to None if not desired
plots_dir = output_dir / "seasonal_diurnal_profiles" / "conus"
# ---------------------------------------------------

if plots_dir is not None:
    make_dir(plots_dir)

options = OptionPresenter(list(model.keys()))
options.present_options(name_func = lambda x: x[1])
input_str = input("Which component's data would you like to plot? ")
component_id = options.get_option(input_str)

seasonal_diurnal_profiles(model, component_id, plots_dir = plots_dir, show=show, 
                          subsector_plot=True, enduse_plot=True, enduse_by_subsector_plots=False,
                          area_plots=True, line_plots=False)

 1: Residential
 2: Commercial
 3: Industrial
 4: Commercial Gaps
 5: Industrial Gaps
 6: Transportation Gaps
 7: Municipal Water
 8: Outdoor Lighting
 9: Distributed Generation
Which component's data would you like to plot? 7


NameError: name 'n_leg_cols' is not defined

## Census Divisions

In [ ]:
# Create the Census Divisions model if it does not exist or overwrite is True
model_name = 'census_division_hourly_residuals' if analysis_type == AnalysisType.Residuals else "dsgrid_site_energy_census_division_hourly"
model_path = output_dir / model_name
if model_path.exists() and overwrite:
    logger.info(f"Deleting {model_path} and re-creating. If this "
                "is not the desired behavior, set overwrite = False.")
    shutil.rmtree(model_path)

if not model_path.exists():
    model = get_model(dsgrid_dataset_path)
    model.map_dimension(model_path, census_divisions, mappings)
model = get_model(model_path)

In [ ]:
geo_enum = model.components[(ComponentType(ComponentType.BOTTOMUP),'Residential')].datafile.geo_enum

options = OptionPresenter(geo_enum.ids)
options.present_options(name_func = geo_enum.get_name)
input_str = input("Which census division would you like to examine? ")
geo_id = options.get_option(input_str)

## States

In [ ]:
model = get_model(dsgrid_dataset_path)

In [ ]:
geo_enum = conus_states

options = OptionPresenter(geo_enum.ids)
options.present_options(name_func = geo_enum.get_name)
input_str = input("Which state would you like to examine? ")
geo_id = options.get_option(input_str)